In [6]:
import tensorflow as tf
path = "/home/tfuser/project/Satelite/data/dataset/train"
data = tf.data.Dataset.load(path)

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from scipy.ndimage import gaussian_filter
import tensorflow as tf
from pathlib import Path
import numpy as np
from helpers.UNET import UNET
from helpers.load import LoadandAugment


# Base path dataset: /home/tfuser/project/Satelite/data/dataset
path = "/home/tfuser/project/Satelite/data/dataset/"
train_path = path + "train"
val_path = path + "val"
test_path = path + "test"

# Load and augment data
# Initialize data loaders
train_data = LoadandAugment(train_path, "train", 8)
val_data = LoadandAugment(val_path, "val", 8)

unet = UNET(input_shape=(1024, 1024, 4))
unet.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
unet.model.fit(train_data, validation_data=val_data, epochs=2)

NameError: name 'mixed_precision' is not defined